In [33]:
import scipy.io as scipy
import os
import os.path as osp
import pandas as pd
import numpy as np
import pickle
import networkx as nx
from tqdm import tqdm

In [54]:
BASEDIR = os.getcwd()
MAP_NAME = 'NYC_Manhattan_Map.mat'
REQ_NAME = 'Requests.mat'
OUTPUT_NAME = 'Manhattan_Map' 

In [55]:
map = scipy.loadmat(osp.join(BASEDIR,MAP_NAME))

In [56]:
map_Arcs = pd.DataFrame(map['Arcs']) #ArcsID, Node_Origin, Node_destination, starts from 1 
map_CityArcs = pd.DataFrame(map['CityArcs']) #NodeID, NodeID, starting from 1, 1 means connected.
map_EdgeTimes = pd.DataFrame(map['EdgeTimes']) #ArcTime in seconds, position is ArcID
map_Nodes = pd.DataFrame(map['Nodes']) # Node ID, latitude, longtitude

In [57]:
# Make the index start from 1
map_CityArcs.index = map_CityArcs.index + 1
map_CityArcs.columns = map_CityArcs.columns + 1

In [58]:
map_Arcs.rename(columns={0:'ArcID',1:'Oid',2:'Did'},inplace=True)
map_EdgeTimes.rename(columns={0:'ArcTime'},inplace=True)
map_Nodes.rename(columns={0:'NodeID',1:'Latitude',2:'Longitude'},inplace=True)

In [59]:
map_Nodes['NodeID'] = map_Nodes['NodeID'].astype(int)
map_Nodes

,NodeID,Latitude,Longitude
0,1,40.706991,-74.017946
1,2,40.706175,-74.017930
2,3,40.707914,-74.017808
3,4,40.706840,-74.017575
4,5,40.707624,-74.017503
...,...,...,...
4086,4087,40.868096,-73.911771
4087,4088,40.872866,-73.911759
4088,4089,40.871035,-73.911682
4089,4090,40.873201,-73.911568


In [60]:
map_Arcs

,ArcID,Oid,Did
0,1,1,4
1,2,2,4
2,3,2,7
3,4,3,6
4,5,4,1
...,...,...,...
9447,9448,4089,4091
9448,9449,4090,4084
9449,9450,4091,4084
9450,9451,4091,4088


In [62]:
# Add self-Connection to map_CityArcs
for i in range(1, map_Nodes.shape[0]+1):
    map_CityArcs.loc[i,i] = 1

map_CityArcs

,1,2,3,4,5,6,7,8,9,10,...,4082,4083,4084,4085,4086,4087,4088,4089,4090,4091
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4089,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,1
4090,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [63]:
map_EdgeTimes.insert(0, "ArcID", range(1, len(map_EdgeTimes) + 1))
map_EdgeTimes

,ArcID,ArcTime
0,1,16
1,2,30
2,3,38
3,4,11
4,5,23
...,...,...
9447,9448,37
9448,9449,93
9449,9450,45
9450,9451,22


In [64]:
# Generate Graph
G = nx.DiGraph()
num_edges = len(map_Arcs) #num of edges in the map
arcs = tqdm(map_Arcs.iterrows(), total=num_edges, ncols=100, desc='Building network...') #ncols is the width of the progress bar

for idx, arcs in arcs:
    Oid = arcs['Oid']
    Did = arcs['Did']
    ArcID = arcs['ArcID']-1 #EdgeTime is indexed from 0
    G.add_node(Oid) #add current node to the graph
    G.add_edge(Oid, Did, TimeCost=map_EdgeTimes.iloc[ArcID]['ArcTime']) #add the arc to the graph

print('Network data loaded.')
print(f'Network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.')

Building network...: 100%|███████████████████████████████████| 9452/9452 [00:00<00:00, 13330.46it/s]

Network data loaded.
Network has 4091 nodes and 9452 edges.


In [67]:
# Find Shortest Path and save to pickel file

print('Computing the shortest path for every node pair')
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

all_path_table = {} 

rng = tqdm(range(1, num_nodes+1), ncols=100, desc='Computing all_path_table')
for node in rng:
    all_path_table[node] = dict(nx.single_source_dijkstra_path(G, node, cutoff=None, weight='TimeCost'))
    path = dict(nx.single_source_dijkstra_path(G, node, cutoff=None, weight='TimeCost'))
    timeCost = dict(nx.single_source_dijkstra_path_length(G, node, cutoff=None, weight='TimeCost'))
    path_timeCost_dict = {k: (path[k], timeCost[k]) for k in path.keys()}
    all_path_table[node] = path_timeCost_dict #store the path and time cost for each node

with open(BASEDIR + '/NYC_Manhattan_AllPathTable.pickle', 'wb') as f:
    pickle.dump(all_path_table, f)

Computing the shortest path for every node pair


Computing all_path_table: 100%|█████████████████████████████████| 4091/4091 [07:10<00:00,  9.50it/s]


In [ ]:
# Compile to All_Path_table
# All_Path_table[Oid][Did][0]: Path
# All_Path_table[Oid][Did][1]: Time

In [56]:
# Load Requests File
req = scipy.loadmat(osp.join(BASEDIR,REQ_NAME))
req_matrix = pd.DataFrame(req['Requests']) #Orid, Did, ReqTime(mins, need to add random secs for this), size of req

In [57]:
req_matrix.rename(columns={0:'Oid',1:'Did',2:'ReqTime',3:'Size'},inplace=True)
req_matrix

,Oid,Did,ReqTime,Size
0,1023,1569,0,6
1,2429,2218,0,1
2,1627,3942,0,5
3,2561,2706,0,1
4,2364,2834,0,2
...,...,...,...,...
10769,1107,2597,59,5
10770,1556,483,59,1
10771,2301,628,59,3
10772,805,1181,59,1


In [58]:
# Convert ReqTime to seconds
req_matrix['ReqTime'] = req_matrix['ReqTime'] * 60

# Generate random numbers between 0 and 59
random_seconds = np.random.randint(0, 60, size=len(req_matrix))

# Add random seconds to ReqTime column
req_matrix['ReqTime'] += random_seconds

req_matrix


,Oid,Did,ReqTime,Size
0,1023,1569,8,6
1,2429,2218,31,1
2,1627,3942,36,5
3,2561,2706,12,1
4,2364,2834,27,2
...,...,...,...,...
10769,1107,2597,3588,5
10770,1556,483,3570,1
10771,2301,628,3568,3
10772,805,1181,3595,1
